In [4]:
import json
import os
from tqdm import tqdm

In [5]:
# !wget https://huggingface.co/datasets/mesolitica/synthetic-multiturn-multimodal/resolve/main/synthetic-multi-audio-relationship.jsonl
# !wget https://huggingface.co/datasets/mesolitica/synthetic-multiturn-multimodal/resolve/main/synthetic-multi-images-multi-audio-relationship.jsonl
# !wget https://huggingface.co/datasets/mesolitica/synthetic-multiturn-multimodal/resolve/main/synthetic-multi-images-relationship.jsonl

In [6]:
en, ms = [], []
with open('synthetic-multi-images-relationship.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        filename = []
        for f in l['filename']:
            f = os.path.join('/home/ubuntu/filtered-blip-images', f)
            filename.append(f)
                
        en.append({
            'filename': filename,
            'conversations': [
                {'role': 'user', 'content': '<image> <image> ' + l['instruction']},
                {'role': 'assistant', 'content': l['answer']},
            ]
        })
        ms.append({
            'filename': filename,
            'conversations': [
                {'role': 'user', 'content': '<image> <image> ' + l['instruction_ms']},
                {'role': 'assistant', 'content': l['answer_ms']},
            ]
        })

100000it [00:01, 59289.96it/s]


In [7]:
len(en)

100000

In [9]:
len(ms)

100000

In [10]:
ms[0]

{'filename': ['/home/ubuntu/filtered-blip-images/00370/003705168.jpg',
  '/home/ubuntu/filtered-blip-images/00611/006111738.jpg'],
 'conversations': [{'role': 'user',
   'content': '<image> <image> Apakah yang berkaitan antara gambar 1 dan gambar 2'},
  {'role': 'assistant',
   'content': 'Tiada hubungan langsung antara Gambar 1 (Paco Rabanne 1 Million Edp Gift Set) dan Gambar 2 (Raspberry Lemonade menjadi must dalam adegan koktel terkenal di New York). Kedua-duanya tidak berkaitan antara satu sama lain. Gambar pertama ialah imej set hadiah minyak wangi, manakala gambar kedua ialah gambaran koktel popular di New York City.'}]}

In [13]:
en[0]

{'filename': ['/home/ubuntu/filtered-blip-images/00370/003705168.jpg',
  '/home/ubuntu/filtered-blip-images/00611/006111738.jpg'],
 'conversations': [{'role': 'user',
   'content': '<image> <image> What is related between picture 1 and picture 2'},
  {'role': 'assistant',
   'content': "There is no direct relation between Picture 1 (Paco Rabanne 1 Million Edp Gift Set) and Picture 2 (Raspberry Lemonade being a must in New York's famous cocktail scene). Both are unrelated to each other. The first picture is an image of a perfume gift set, while the second picture is a depiction of a popular cocktail in New York City."}]}

In [11]:
with open('prepared-relationship-en.jsonl', 'w') as fopen:
    for d in en:
        fopen.write(f'{json.dumps(d)}\n')

In [12]:
with open('prepared-relationship-ms.jsonl', 'w') as fopen:
    for d in ms:
        fopen.write(f'{json.dumps(d)}\n')